In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**-----------Load The Data------------**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, MQ)
]

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


-----------

**--------QuickSelect Algorithm----------------**

In [ ]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

---------

**----------KD Tree Dyn---------------------**

In [ ]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**-------KD Tree Node Size Optimization For Rebuild-------**

**--------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2_500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 881.22it/s]


-------------------------------------------------------------
Build Time: 28.8871 sec, Search Time: 0.7328 sec
Leaf Size: 50, Total Time: 29.6199 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 1045.26it/s]


-------------------------------------------------------------
Build Time: 26.0389 sec, Search Time: 0.6336 sec
Leaf Size: 100, Total Time: 26.6726 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 17.7052 sec, Search Time: 0.6532 sec
Leaf Size: 2,400, Total Time: 18.3584 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 822.41it/s]


-------------------------------------------------------------
Build Time: 18.2917 sec, Search Time: 0.6162 sec
Leaf Size: 2,450, Total Time: 18.9078 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 698.15it/s]

-------------------------------------------------------------
Build Time: 17.5003 sec, Search Time: 0.6574 sec
Leaf Size: 2,500, Total Time: 18.1577 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 28.8871 sec, Search Time: 0.7328 sec, Total Time: 29.6199 sec
Leaf Size: 100, Build Time: 26.0389 sec, Search Time: 0.6336 sec, Total Time: 26.6726 sec
Leaf Size: 150, Build Time: 24.6174 sec, Search Time: 0.6572 sec, Total Time: 25.2746 sec
Leaf Size: 200, Build Time: 23.8826 sec, Search Time: 0.7220 sec, Total Time: 24.6046 sec
Leaf Size: 250, Build Time: 22.9982 sec, Search Time: 0.6436 sec, Total Time: 23.6418 sec
Leaf Size: 300, Build Time: 22.5172 sec, Search Time: 0.6193 sec, Total Time: 23.1365 sec
Leaf Size: 350, Build Time: 21.9487 sec, Search Time: 0.6589 sec, Total Time: 22.6076 sec
Leaf Size: 400, Build Time: 23.3056 sec, Search Time: 0.6463 sec, Total Time: 23.9519 sec
Leaf Size: 450, Build Time: 22.7169 sec, Search Time: 0.6544 sec, Total Time: 23.3713

**-------------Run 2-----------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 2500  # Ensure inclusive range
leaf_size_step = 50

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 768.25it/s]


-------------------------------------------------------------
Build Time: 28.2476 sec, Search Time: 0.7676 sec
Leaf Size: 50, Total Time: 29.0152 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 978.97it/s]


-------------------------------------------------------------
Build Time: 26.4388 sec, Search Time: 0.6993 sec
Leaf Size: 100, Total Time: 27.1382 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 18.0920 sec, Search Time: 0.6489 sec
Leaf Size: 2,400, Total Time: 18.7409 sec

49: === Testing with Leaf Size: 2,450, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 769.22it/s]


-------------------------------------------------------------
Build Time: 18.3431 sec, Search Time: 0.6572 sec
Leaf Size: 2,450, Total Time: 19.0004 sec

50: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 766.54it/s]

-------------------------------------------------------------
Build Time: 18.1032 sec, Search Time: 0.6388 sec
Leaf Size: 2,500, Total Time: 18.7421 sec

=== Summary of KD Tree Node Size Test ===
Leaf Size: 50, Build Time: 28.2476 sec, Search Time: 0.7676 sec, Total Time: 29.0152 sec
Leaf Size: 100, Build Time: 26.4388 sec, Search Time: 0.6993 sec, Total Time: 27.1382 sec
Leaf Size: 150, Build Time: 24.2386 sec, Search Time: 0.6695 sec, Total Time: 24.9081 sec
Leaf Size: 200, Build Time: 23.8116 sec, Search Time: 0.6277 sec, Total Time: 24.4393 sec
Leaf Size: 250, Build Time: 22.8876 sec, Search Time: 0.6261 sec, Total Time: 23.5137 sec
Leaf Size: 300, Build Time: 22.6621 sec, Search Time: 0.6108 sec, Total Time: 23.2729 sec
Leaf Size: 350, Build Time: 21.9924 sec, Search Time: 0.6041 sec, Total Time: 22.5965 sec
Leaf Size: 400, Build Time: 22.7916 sec, Search Time: 0.6068 sec, Total Time: 23.3984 sec
Leaf Size: 450, Build Time: 22.1667 sec, Search Time: 0.6566 sec, Total Time: 22.8234

**-------------Opt Node Size For Rebuild-------------**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700,
  750, 800, 850, 900, 950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
 1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050,2100,
 2150,2200,2250,2300,2350,2400,2450,2500])

# Total times from two runs (or versions)
total_times_1 = np.array([29.6199,26.6726,25.2746,24.6046,23.6418,23.1365,22.6076,23.9519,23.3713,
 22.0789,22.272 ,21.803 ,22.0911,21.6514,22.1628,22.4267,21.6203,21.2486,
 21.7261,20.2991,20.5823,19.8766,19.8519,20.1735,19.8991,19.8023,20.4149,
 19.8285,19.2107,20.2885,20.7079,20.3559,21.0756,19.4659,19.3212,19.5821,
 20.0063,19.9428,19.4656,18.2919,18.4629,18.6592,17.7393,18.235 ,18.5362,
 18.816 ,18.2054,18.3584,18.9078,18.1577])

total_times_2 = np.array([29.0152,27.1382,24.9081,24.4393,23.5137,23.2729,22.5965,23.3984,22.8234,
 21.5875,22.2976,21.6182,21.5877,21.6828,21.8536,21.8213,21.3202,21.5198,
 22.263 ,19.7658,20.0378,20.2073,20.0288,19.995 ,20.0568,20.0258,20.0562,
 20.4651,19.925 ,19.9977,20.838 ,20.2759,19.5946,20.4554,20.1006,20.2103,
 20.1907,19.9932,19.6769,18.7187,18.6616,18.8981,18.3962,19.2069,18.9926,
 19.2472,18.9178,18.7409,19.0004,18.7421])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 18.0678
Minimum found at leaf size: 2150


--------------------------------

**-----Epsilon Optimization For Rebuild------**

**-----------Run 1-------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2150
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.05  # Ensure inclusive range
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 725.82it/s]


-------------------------------------------------------------
Build Time: 19.3300 sec, Search Time: 0.6778 sec
Epsilon: 0.0, Total Time: 20.0078 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:01<00:00, 58.38it/s]


-------------------------------------------------------------
Build Time: 12.3349 sec, Search Time: 2.1064 sec
Epsilon: 0.001, Total Time: 14.4413 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 5.4665 sec, Search Time: 4.8504 sec
Epsilon: 0.048, Total Time: 10.3169 sec

50: === Testing with Epsilon: 0.049, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.049): 100%|██████████| 100/100 [00:04<00:00, 21.94it/s]


-------------------------------------------------------------
Build Time: 5.5634 sec, Search Time: 4.7673 sec
Epsilon: 0.049, Total Time: 10.3307 sec

51: === Testing with Epsilon: 0.05, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.05): 100%|██████████| 100/100 [00:04<00:00, 21.57it/s]

-------------------------------------------------------------
Build Time: 5.4102 sec, Search Time: 4.8434 sec
Epsilon: 0.05, Total Time: 10.2536 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 19.3300 sec, Search Time: 0.6778 sec, Total Time: 20.0078 sec
Epsilon: 0.001, Build Time: 12.3349 sec, Search Time: 2.1064 sec, Total Time: 14.4413 sec
Epsilon: 0.002, Build Time: 12.0858 sec, Search Time: 2.2775 sec, Total Time: 14.3633 sec
Epsilon: 0.003, Build Time: 10.5252 sec, Search Time: 2.2790 sec, Total Time: 12.8042 sec
Epsilon: 0.004, Build Time: 10.2413 sec, Search Time: 2.3810 sec, Total Time: 12.6223 sec
Epsilon: 0.005, Build Time: 9.9731 sec, Search Time: 2.4680 sec, Total Time: 12.4411 sec
Epsilon: 0.006, Build Time: 9.7089 sec, Search Time: 2.6027 sec, Total Time: 12.3116 sec
Epsilon: 0.007, Build Time: 9.5389 sec, Search Time: 3.1073 sec, Total Time: 12.6462 sec
Epsilon: 0.008, Build Time: 8.8345 sec, Search Time: 2.6665 sec, Total Time: 11.5011 sec
E

**----------Run 2--------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size = 2150
# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.05  # Ensure inclusive range
epsilon_size_step = 0.001

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the KD Tree ---
    build_start = time.perf_counter()
    kdtree = KDTree(leaf_size=leaf_size, epsilon=epsilon)
    kdtree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:00<00:00, 791.74it/s]


-------------------------------------------------------------
Build Time: 18.6870 sec, Search Time: 0.6767 sec
Epsilon: 0.0, Total Time: 19.3637 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:01<00:00, 60.35it/s]


-------------------------------------------------------------
Build Time: 12.0096 sec, Search Time: 2.0588 sec
Epsilon: 0.001, Total Time: 14.0685 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 5.2763 sec, Search Time: 4.5450 sec
Epsilon: 0.048, Total Time: 9.8213 sec

50: === Testing with Epsilon: 0.049, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.049): 100%|██████████| 100/100 [00:04<00:00, 22.92it/s]


-------------------------------------------------------------
Build Time: 5.4090 sec, Search Time: 4.5748 sec
Epsilon: 0.049, Total Time: 9.9838 sec

51: === Testing with Epsilon: 0.05, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.05): 100%|██████████| 100/100 [00:04<00:00, 22.21it/s]

-------------------------------------------------------------
Build Time: 5.4824 sec, Search Time: 4.7133 sec
Epsilon: 0.05, Total Time: 10.1956 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 18.6870 sec, Search Time: 0.6767 sec, Total Time: 19.3637 sec
Epsilon: 0.001, Build Time: 12.0096 sec, Search Time: 2.0588 sec, Total Time: 14.0685 sec
Epsilon: 0.002, Build Time: 11.4811 sec, Search Time: 2.1269 sec, Total Time: 13.6080 sec
Epsilon: 0.003, Build Time: 10.3066 sec, Search Time: 2.1686 sec, Total Time: 12.4752 sec
Epsilon: 0.004, Build Time: 10.0191 sec, Search Time: 2.3503 sec, Total Time: 12.3694 sec
Epsilon: 0.005, Build Time: 9.7915 sec, Search Time: 2.3697 sec, Total Time: 12.1612 sec
Epsilon: 0.006, Build Time: 9.6744 sec, Search Time: 2.4774 sec, Total Time: 12.1519 sec
Epsilon: 0.007, Build Time: 9.5067 sec, Search Time: 2.5078 sec, Total Time: 12.0145 sec
Epsilon: 0.008, Build Time: 8.8137 sec, Search Time: 2.6526 sec, Total Time: 11.4663 sec
E

**--------Epsilon Opt Value For Rebuild------------**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ])

# Total times from two runs (or versions)
total_times_1 = np.array([20.0078,14.4413,14.3633,12.8042,12.6223,12.4411,12.3116,12.6462,11.5011,
 11.5109,11.5522,11.3355,11.4393,11.2665,11.1061,11.2555,10.5206,10.6304,
 11.1294,10.8353,12.0259,11.8676,11.869 ,11.9335,11.9487,11.7811,11.6017,
 11.3185,11.6703,10.3277,11.7463,11.8199,11.7203,11.446 ,11.6212,11.4434,
 10.6776,10.655 ,10.8092,10.6179,10.6157,10.4758,10.6592,10.5739,10.7003,
 10.5226,10.771 ,10.2248,10.3169,10.3307,10.2536])

total_times_2 = np.array([19.3637,14.0685,13.608 ,12.4752,12.3694,12.1612,12.1519,12.0145,11.4663,
 11.3345,11.3263,11.0641,10.9428,10.749 ,10.7816,10.8332,10.4921,10.4403,
 10.5085,10.5993,10.5046,10.943 ,10.6781,10.4671,10.6941,10.2616,10.0507,
 10.5109,10.043 ,10.0001, 9.9698, 9.7563, 9.8568, 9.8546, 9.5806, 9.7004,
 10.0036, 9.9311,10.21  ,10.4175,10.0699,10.5046,10.2678,10.3652, 9.9629,
 10.3356, 9.992 , 9.8496, 9.8213, 9.9838,10.1956])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 10.0372
Minimum found at epsilon size: 0.047


--------------------

**------------Epsilon Optimization for Insertion-------------**

**-------------------Run 1---------------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.05  # Ensure inclusive range
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:02<00:00, 34.74it/s]


-------------------------------------------------------------
Build Time: 8.5215 sec, Search Time: 3.1745 sec
Epsilon: 0.0, Total Time: 11.6960 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:02<00:00, 34.97it/s]


-------------------------------------------------------------
Build Time: 8.2282 sec, Search Time: 3.1642 sec
Epsilon: 0.001, Total Time: 11.3923 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.1998 sec, Search Time: 3.5341 sec
Epsilon: 0.048, Total Time: 11.7338 sec

50: === Testing with Epsilon: 0.049, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.049): 100%|██████████| 100/100 [00:03<00:00, 32.49it/s]


-------------------------------------------------------------
Build Time: 7.9510 sec, Search Time: 3.5969 sec
Epsilon: 0.049, Total Time: 11.5479 sec

51: === Testing with Epsilon: 0.05, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.05): 100%|██████████| 100/100 [00:02<00:00, 33.40it/s]

-------------------------------------------------------------
Build Time: 8.3565 sec, Search Time: 3.4841 sec
Epsilon: 0.05, Total Time: 11.8406 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.5215 sec, Search Time: 3.1745 sec, Total Time: 11.6960 sec
Epsilon: 0.001, Build Time: 8.2282 sec, Search Time: 3.1642 sec, Total Time: 11.3923 sec
Epsilon: 0.002, Build Time: 8.1151 sec, Search Time: 3.0908 sec, Total Time: 11.2059 sec
Epsilon: 0.003, Build Time: 7.5975 sec, Search Time: 3.1642 sec, Total Time: 10.7616 sec
Epsilon: 0.004, Build Time: 8.1137 sec, Search Time: 3.1436 sec, Total Time: 11.2573 sec
Epsilon: 0.005, Build Time: 8.1177 sec, Search Time: 3.2097 sec, Total Time: 11.3274 sec
Epsilon: 0.006, Build Time: 8.0883 sec, Search Time: 3.3392 sec, Total Time: 11.4275 sec
Epsilon: 0.007, Build Time: 8.3664 sec, Search Time: 3.3609 sec, Total Time: 11.7272 sec
Epsilon: 0.008, Build Time: 8.2152 sec, Search Time: 3.4192 sec, Total Time: 11.6344 sec
Epsilo

**----------------Run 2-------------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for KD Tree
results = []

leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

# Define epsilon range
epsilon_range_start = 0.0
epsilon_range_end = 0.05  # Ensure inclusive range
epsilon_size_step = 0.001

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Build the tree with 100K points, using the optimized n_s, and epsilon
    kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    kdtree.build(current_data_rebuild)

    # Readjust for the insertion
    data_size, query_size = test_combinations[0]
    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    kdtree.epsilon = epsilon

    # --- Build the KD Tree With Insertions---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting KD Tree"):
        kdtree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = kdtree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of KD Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.0): 100%|██████████| 100/100 [00:02<00:00, 35.09it/s]


-------------------------------------------------------------
Build Time: 8.1563 sec, Search Time: 3.3233 sec
Epsilon: 0.0, Total Time: 11.4796 sec

2: === Testing with Epsilon: 0.001, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.001): 100%|██████████| 100/100 [00:02<00:00, 34.05it/s]


-------------------------------------------------------------
Build Time: 8.1501 sec, Search Time: 3.3915 sec
Epsilon: 0.001, Total Time: 11.5416 sec

3: === Testing with Epsilon: 0.002, Data Size: 1,000,000 points, Query Size: 100 queries ===


...

-------------------------------------------------------------
Build Time: 8.3239 sec, Search Time: 3.5663 sec
Epsilon: 0.048, Total Time: 11.8902 sec

50: === Testing with Epsilon: 0.049, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.049): 100%|██████████| 100/100 [00:03<00:00, 32.27it/s]


-------------------------------------------------------------
Build Time: 8.0678 sec, Search Time: 3.5984 sec
Epsilon: 0.049, Total Time: 11.6663 sec

51: === Testing with Epsilon: 0.05, Data Size: 1,000,000 points, Query Size: 100 queries ===


Querying (Epsilon 0.05): 100%|██████████| 100/100 [00:03<00:00, 31.79it/s]

-------------------------------------------------------------
Build Time: 8.1232 sec, Search Time: 3.6583 sec
Epsilon: 0.05, Total Time: 11.7815 sec

=== Summary of KD Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.1563 sec, Search Time: 3.3233 sec, Total Time: 11.4796 sec
Epsilon: 0.001, Build Time: 8.1501 sec, Search Time: 3.3915 sec, Total Time: 11.5416 sec
Epsilon: 0.002, Build Time: 7.9046 sec, Search Time: 3.4183 sec, Total Time: 11.3229 sec
Epsilon: 0.003, Build Time: 8.2059 sec, Search Time: 3.3360 sec, Total Time: 11.5419 sec
Epsilon: 0.004, Build Time: 8.1665 sec, Search Time: 3.4457 sec, Total Time: 11.6122 sec
Epsilon: 0.005, Build Time: 8.2227 sec, Search Time: 3.8394 sec, Total Time: 12.0621 sec
Epsilon: 0.006, Build Time: 8.5525 sec, Search Time: 3.5904 sec, Total Time: 12.1429 sec
Epsilon: 0.007, Build Time: 8.1632 sec, Search Time: 3.5978 sec, Total Time: 11.7610 sec
Epsilon: 0.008, Build Time: 8.3292 sec, Search Time: 3.5018 sec, Total Time: 11.8310 sec
Epsilo

**------Epsilon Optimization For Insertion----**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([0.   ,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01 ,0.011,
 0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02 ,0.021,0.022,0.023,
 0.024,0.025,0.026,0.027,0.028,0.029,0.03 ,0.031,0.032,0.033,0.034,0.035,
 0.036,0.037,0.038,0.039,0.04 ,0.041,0.042,0.043,0.044,0.045,0.046,0.047,
 0.048,0.049,0.05 ])

# Total times from two runs (or versions)
total_times_1 = np.array([11.696 ,11.3923,11.2059,10.7616,11.2573,11.3274,11.4275,11.7272,11.6344,
 11.8356,11.4339,11.454 ,11.8898,11.4723,11.702 ,11.5116,11.7852,11.5904,
 11.4477,11.5564,11.5241,11.7251,11.8038,11.8349,11.3382,11.3123,11.577 ,
 11.9591,11.8066,11.3493,11.4982,11.421 ,11.3506,11.8616,11.5809,11.609 ,
 11.6827,11.9086,11.5423,11.6636,11.3367,11.4474,11.6661,12.0212,11.3978,
 11.4942,11.4259,11.6793,11.7338,11.5479,11.8406])

total_times_2 = np.array([11.4796,11.5416,11.3229,11.5419,11.6122,12.0621,12.1429,11.761 ,11.831 ,
 11.667 ,11.9101,11.6727,11.5887,11.8194,11.5402,11.4397,11.7711,11.7265,
 11.4755,11.9599,11.5244,11.7692,11.4946,11.6027,11.4684,11.792 ,11.7678,
 11.4826,12.0996,11.7615,11.8293,11.6173,11.6673,11.6697,11.7068,11.8402,
 11.8405,11.7974,11.7251,11.9747,12.3998,11.4984,11.9381,11.562 ,11.7945,
 12.1702,11.5833,11.3355,11.8902,11.6663,11.7815])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 13.2479
Minimum found at epsilon size: 0.04
